<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Symbol_to_Symbol-1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbol-to_Symbol Attack

# Loading Drive and folders

In [1]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [2]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Loading Necesary Libraries

In [10]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

# Generate Train/Test data

## Old input data

In [49]:
input_data_path=path+"/rawData"

distances=[i for i in range(1,26)]
nsamples=50
span_length=80
nsymbols=2048

######
min_dist=0
max_dist=3000
selCP=[1,7,10,15]
selCP_pos=[(-3,3),(1,1),(-1,-1),(1,-3)]
my_centers=[[-3,3],[-1,3],[1,3],[3,3],[-3,1],[-1,1],[1,1],[3,1],[-3,-1],[-1,-1],[1,-1],[3,-1], [-3,-3],[-1,-3],[1,-3],[3,-3]]
######

In [50]:
def create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  X=None
  Y=[]
  colnames=['i'+str(i) for i in range(nsymbols)]

  for d in distances:
    dist=d*span_length
    if dist<min_dist or dist>max_dist: continue
    filename='consts_'+str(d)+'span.csv'
    df_aux=pd.read_csv(input_data_path+'/'+filename, sep=",", header=None)
    df_aux = df_aux.T
    df_aux.columns=colnames
    Y=Y+[dist]*df_aux.shape[0]
    if X is None: X=df_aux
    else: X=X.append(df_aux)
  return X, Y

In [51]:
def strToTuple(s):
    s_aux=s.split("i")
    s=s_aux[0]+"j"
    return complex(s)

In [52]:
def test_train_indexes_v1():
  train_idxs = []
  test_idxs = []
  for i in range(25):
      for j in range(25):
          train_idxs.append(50*i + j)
          test_idxs.append(50*(i+1)-1-j)
  return train_idxs, test_idxs

In [53]:
def test_train_indexes_v2():
  train_idxs2 = []
  test_idxs2 = []
  for i in range(25):
      for j in range(25):
          test_idxs2.append(50*(i+1)-1-j)

  for i in range(25):
    for j in range(2):
      train_idxs2.append(50*i + j)
  return train_idxs2, test_idxs2

## Test Old

In [54]:
# Obtain df for each distance (X) and  distance*nsymbols (Y)
X, Y = create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000)

In [55]:
X = X.applymap(strToTuple)

In [56]:
train_idxs, test_idxs = test_train_indexes_v1()
train_idxs2, test_idxs2 = test_train_indexes_v2()

X_train = X.iloc[train_idxs].reset_index(drop = "True")
X_test = X.iloc[test_idxs].reset_index(drop = "True")
X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
X_test2 = X.iloc[test_idxs2].reset_index(drop = "True")

In [57]:
data = {}

for i in range(0,25):
  data[str(80*(i+1))] = {"Train":X_train.iloc[i*25:(i*25+25)].reset_index(drop = "True"),"Test":X_test.iloc[i*25:(i*25+25)].reset_index(drop = "True")}

data_new = {}

for i in range(0,25):
  data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}

## New input data

In [58]:
def create_df(input_data_path):
  return pd.read_excel(input_data_path, sheet_name = "Sheet1")

def strToTuple(s):
    s_aux=s.split("i")
    s=s_aux[0]+"j"
    return complex(s)

def test_train_indexes_v1(df, distances):
  train_idxs = []
  test_idxs = []

  amt_of_distances = len(distances)
  total_of_samples = df.shape[0]

  amt_of_sample_per_distance = int(total_of_samples/amt_of_distances)
  half_of_sample_per_distance = int(amt_of_sample_per_distance/2)

  indx_cnt = 0
  for dist in distances:

    for row_index in range(amt_of_sample_per_distance):
      # Select the first 5 samples as train
      if row_index < half_of_sample_per_distance:
        train_idxs.append(indx_cnt)
        indx_cnt += 1
      else:
        test_idxs.append(indx_cnt)
        indx_cnt += 1

  return train_idxs, test_idxs

def test_train_indexes_v2(df, distances):
  train_idxs2 = []
  test_idxs2 = []

  amt_of_distances = len(distances)
  total_of_samples = df.shape[0]

  amt_of_sample_per_distance = int(total_of_samples/amt_of_distances)
  half_of_sample_per_distance = int(amt_of_sample_per_distance/2)

  indx_cnt = 0
  for dist in distances:

    for row_index in range(amt_of_sample_per_distance):
      # Select the first 5 samples as train
      if row_index == 1 or row_index == 2:
        train_idxs2.append(indx_cnt)
        indx_cnt += 1
      elif row_index >= half_of_sample_per_distance:
        test_idxs2.append(indx_cnt)
        indx_cnt += 1
      else:
        indx_cnt += 1

  return train_idxs2, test_idxs2

def create_data_dict(X_train, X_test, X_train2, X_test2, distances):

  data = {}
  data_new = {}
  data_full = {}
  data_new_full = {}

  for dist in distances:

    X_train_n = X_train[X_train['Distance_km'].isin([int(dist)])]
    X_test_n = X_test[X_test['Distance_km'].isin([int(dist)])]

    data_full[str(dist)] = {'Train':X_train_n.reset_index(drop = 'True'),'Test':X_test_n.reset_index(drop = 'True')}

    X_train_nn = X_train_n.iloc[:, 5:X_train_n.shape[1]]
    X_test_nn = X_test_n.iloc[:, 5:X_test_n.shape[1]]

    X_train2_n = X_train2[X_train2['Distance_km'].isin([int(dist)])]
    X_test2_n = X_test2[X_test2['Distance_km'].isin([int(dist)])]

    data_new_full[str(dist)] = {'Train':X_train2_n.reset_index(drop = 'True'),'Test':X_test2_n.reset_index(drop = 'True')}

    X_train2_nn = X_train2_n.iloc[:, 5:X_train2_n.shape[1]]
    X_test2_nn = X_test2_n.iloc[:, 5:X_test2_n.shape[1]]

    data[str(dist)] = {'Train':X_train_nn.reset_index(drop = 'True'),'Test':X_test_nn.reset_index(drop = 'True')}
    data_new[str(dist)] = {'Train':X_train2_nn.reset_index(drop = 'True'),'Test':X_test2_nn.reset_index(drop = 'True')}

  return data, data_new, data_full, data_new_full

In [59]:
def get_train_test_data(constellation_path):
  df = create_df(constellation_path)

  # Open df by selected powers
  powers = list(df['power_dBm'].unique())
  for power in powers:

    new_df = df[df['power_dBm'].isin([int(power)])]

    distances = list(new_df['Distance_km'].unique())

    # Data columns 
    X1 = new_df.iloc[:, 5:(new_df.shape[1])]
    X = X1.applymap(strToTuple)

    # Add back extra info columns
    info_df = new_df.iloc[:, 0:5]
    X = pd.merge(info_df, X, left_index=True, right_index=True)


    train_idxs, test_idxs = test_train_indexes_v1(X, distances)
    train_idxs2, test_idxs2 = test_train_indexes_v2(X, distances)
    
    X_train = X.iloc[train_idxs, :].reset_index(drop = "True")
    X_test = X.iloc[test_idxs, :].reset_index(drop = "True")

    X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
    X_test2 = X.iloc[test_idxs2].reset_index(drop = "True")

    data, data_new, data_full, data_new_full = create_data_dict(X_train, X_test, X_train2, X_test2, distances)
    
    return data, data_new, data_full, data_new_full

## Test new data train test

In [60]:
#const_path = path + '/Constellation/QPSK/A.10hops100km/dataSet_Constellation_Samples_QPSK_37GHz_LongHaul_output_10x100.xlsx'
const_path = path + '/Constellation/QPSK/A.10hops100km/const_dummy_QPSK.xlsx'

# Open df by selected powers

df = create_df(const_path)

In [61]:
powers = list(df['power_dBm'].unique())
for power in powers:

  new_df = df[df['power_dBm'].isin([int(power)])]

  distances = list(new_df['Distance_km'].unique())

  # Data columns 
  X1 = new_df.iloc[:, 5:(new_df.shape[1])]
  X = X1.applymap(strToTuple)

  # Add back extra info columns
  info_df = new_df.iloc[:, 0:5]
  X = pd.merge(info_df, X, left_index=True, right_index=True)


  train_idxs, test_idxs = test_train_indexes_v1(X, distances)
  train_idxs2, test_idxs2 = test_train_indexes_v2(X, distances)
  
  X_train = X.iloc[train_idxs, :].reset_index(drop = "True")
  X_test = X.iloc[test_idxs, :].reset_index(drop = "True")

  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True")

  data_x, data_new_x, data_full, data_new_full = create_data_dict(X_train, X_test, X_train2, X_test2, distances)

In [62]:
print(data.keys())
#print(data['100'])
#print(data['100']['Train'])

dict_keys(['80', '160', '240', '320', '400', '480', '560', '640', '720', '800', '880', '960', '1040', '1120', '1200', '1280', '1360', '1440', '1520', '1600', '1680', '1760', '1840', '1920', '2000'])


# New Get Train Test Data

In [63]:
#const_path = path + '/Constellation/QPSK/A.10hops100km/dataSet_Constellation_Samples_QPSK_37GHz_LongHaul_output_10x100.xlsx'
constellation_path = path + '/Constellation/QPSK/A.10hops100km/const_dummy_QPSK.xlsx'

data_x, data_new_x, data_full, data_new_full = get_train_test_data(constellation_path)

# Utils

In [64]:
def grad_b (M,m_og,m_tg,b,beta):
  return 2*beta*(M @ m_og + b - m_tg)

In [65]:
def grad_M(M,cov_og,cov_tg,alpha):
    error = M @ cov_og @ np.transpose(M) - cov_tg  

    m1 = (2*error[0][0] * (2*cov_og[0][0]*M[0][0] + 2*cov_og[0][1]*M[0][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[1][0] + cov_og[0][1]*M[1][1]))
    
    m2 = (2*error[0][0] * (2*cov_og[1][1]*M[0][1] + 2*cov_og[0][1]*M[0][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[1][0] + cov_og[1][1]*M[1][1]))
    
    m3 = (2*error[1][1] * (2*cov_og[0][0]*M[1][0] + 2*cov_og[0][1]*M[1][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[0][0] + cov_og[0][1]*M[0][1]))

    m4 = (2*error[1][1] * (2*cov_og[0][0]*M[1][1] + 2*cov_og[0][1]*M[1][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[0][0] + cov_og[1][1]*M[0][1]))
    
    return alpha*np.array([[m1, m2], [m3, m4]])

In [66]:
def compute_mean_and_cov(data):
  aux_x = [] # Reales
  aux_y = [] # Imag
  for obs in data:
    aux_x.append(obs[0])
    aux_y.append(obs[1])
  return np.array([[np.mean(aux_x)],[np.mean(aux_y)]]), np.cov(aux_x,aux_y)


def total_loss(m_mod,m_tg,cov_mod,cov_tg):
  return (sum(sum(np.power(cov_mod-cov_tg,2))) + sum(np.power(m_mod - m_tg,2)))[0]


def loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b):
  first = alpha*sum(sum(np.power(M @ cov_og @ np.transpose(M) - cov_tg, 2)))
  second = beta*sum(np.power(M @ m_og + b - m_tg, 2))
  a = first+second
  return a

In [67]:
def gradient_descent(alpha,beta,m_tg,m_og,cov_tg,cov_og,nu,log):
  Ms = []
  bs = []
  M = np.random.rand(2,2)
  #M = np.array([[1, 0], [0, 1]])
  b = np.random.rand(2,1)
  #b = np.array([[0], [0]])
  #for i in range(100000):
  i = 0
  while True:
    # print('From:', M @ cov_source @ np.transpose(M))
    # print('To:', cov_target)
    L = loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b) 
    if  L < 1e-20:
        Ms.append(M)
        bs.append(b)
        break
    b = b - nu*grad_b(M,m_og,m_tg,b,beta)
    M = M - nu*grad_M(M,cov_og,cov_tg,alpha)
    i+= 1
    if (i>= 5000 and i <= 5025) or (i>= 5975 and i <= 6000):
      Ms.append(M)
      bs.append(b)
    if not i%5000 and log: print(L)
  if log: print("-"*25)
  return Ms,bs

In [68]:
def generate_comparison_plot(mod_points,ptarget, return_plots=False):
  x1 = [x[0] for  x in mod_points]
  y1 = [x[1] for  x in mod_points]
  x2 = [x[0] for  x in ptarget]
  y2 = [x[1] for  x in ptarget]

  if return_plots:
      return go.Scatter(x = x1, y = y1, mode='markers'), go.Scatter(x = x2, y = y2,mode='markers') 

  fig = make_subplots(rows=1, cols=2)
  fig.add_trace(
    go.Scatter(x = x1, y = y1,mode='markers'),
    
    row=1, col=1
  )
  fig.add_trace(
    go.Scatter(x = x2, y = y2,mode='markers'),
    row=1, col=2
  )
  fig.update_layout(height=500, width=1000, title_text="Point comparison",autosize = False)
  fig.show()

In [69]:
def compute_parameters (const,method,source,target,log = False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  q_target = []
  for i in range(len(source)):
    q_source += [[x.real,x.imag] for x in source.values.tolist()[i] if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]
    q_target += [[x.real,x.imag] for x in target.values.tolist()[i]if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]

  unbiased = (len(q_source)/(len(q_source)-1) * 125/126)

  mean_source,cov_source = compute_mean_and_cov(q_source)
  mean_target, cov_target= compute_mean_and_cov(q_target)

  cov_source *= unbiased
  cov_target *= unbiased

  if method == "GD":
    alpha = 3/4
    beta = 1/4
    M, b = gradient_descent(alpha,beta,mean_target,mean_source,cov_target,cov_source, 0.5,log)
    return M,b

  elif method == "Z":
    return mean_source,cov_source,mean_target,cov_target

In [70]:
def modify_const_GD(const,source,M,b, target = None, return_plots=False):

  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]
      
  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  for mult in mults:
    res = (M @ mult + b).tolist()
    new_points.append([res[0][0],res[1][0]])
  
  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      return new_points, q_target

  return new_points,indexes

In [71]:
def modify_const_Z(const,source,mean_source,cov_source,mean_target,cov_target,target = None, return_plots=False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]

  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  w, v = np.linalg.eig(cov_source)
  S1 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)
  w, v = np.linalg.eig(cov_target)
  S2 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)

  for mult in mults:
    normalized = np.linalg.inv(S1) @ (mult - mean_source)
    denormalized = S2 @ normalized + mean_target

    new_points.append([denormalized[0][0], denormalized[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      

  return new_points, indexes

# Obtain parameters and GD

In [79]:
distances=[i for i in range(1,26)]
M,b = compute_parameters(1,"GD",data[str(distances[0]*80)]['Train'],data[str(distances[24]*80)]['Train'], log = True)

#a,b = modify_const_GD(1,data[str(distances[0]*80)]['Train'],M[-1],b[-1],data[str(distances[24]*80)]['Train'], return_plots=True)

[0.00055049]
[1.54932795e-08]
[2.51436977e-13]
[4.07045888e-18]
-------------------------


In [73]:
M,b = compute_parameters(1,"GD",data_x[str(0)]['Train'],data_x[str(200)]['Train'], log = True)

[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]


KeyboardInterrupt: ignored

# Loss

In [ ]:
import pickle 

with open('b.pickle', 'wb') as handle:
    pickle.dump(b, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
points = []
id = [np.identity(2)]
zero = [np.zeros((2, 1))]
Ms = id + M
bs = zero + b

for i in range(len(Ms)):
  p,_ =  modify_const_GD(1,data[str(distances[0]*80)]['Train'],Ms[i],bs[i])
  points.append(p)

In [ ]:
obs = points[-1]
x = [a[0] for a in obs]
y = [a[1] for a in obs]

fig = px.scatter(x=x, y=y, labels=dict(x="", y = ""))

fig.update_traces(marker=dict(
        color='#007bbf', size=10))

fig.update_layout(width=550, height = 500)
fig.update_yaxes(range = [1.5,4.5])
fig.update_xaxes(range = [-4.5,-1.5])

fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


x1 = [10,8,13,9,11,14,6,4,12,7,5]
x2 = x1
x3 = x1
x4 = [8,8,8,8,8,8,8,19,8,8,8]
y1 = [10.0,6.95,7.58,8.81,8.33,9.96,7.24,4.26,10.84,4.82,5.68]
y2 = [9.14,8.14,8.74,8.77,9.26,8.10,6.13,3.10,9.13,7.26,4.74]
y3 = [7.46,6.77,12.74,7.11,7.81,8.84,6.08,5.39,8.15,6.42,5.73]
y4 = [6.58,5.76,7.71,8.84,8.47,7.04,5.25,12.50,5.56,7.91,6.89]

reg_x = [0,20]
reg_y = [3,13]

fig = make_subplots(rows=2, cols=2)


fig.add_trace(
  go.Scatter(x = reg_x, y = reg_y,mode='lines'),
  
  row=1, col=1
)

fig.add_trace(
  go.Scatter(x = reg_x, y = reg_y,mode='lines'),
  
  row=1, col=2
)

fig.add_trace(
  go.Scatter(x = reg_x, y = reg_y,mode='lines'),
  
  row=2, col=1
)

fig.add_trace(
  go.Scatter(x = reg_x, y = reg_y,mode='lines'),
  
  row=2, col=2
)

fig.add_trace(
  go.Scatter(x = x1, y = y1,mode='markers'),
  
  row=1, col=1
)


fig.add_trace(
  go.Scatter(x = x2, y = y2,mode='markers'),
  row=1, col=2
)
fig.add_trace(
go.Scatter(x = x3, y = y3,mode='markers'),
row=2, col=1
)
fig.add_trace(
go.Scatter(x = x4, y = y4,mode='markers'),
row=2, col=2
)

fig.update_xaxes(range=[0, 20])
fig.update_yaxes(range=[0, 15], dtick = 3)
fig.update_layout(height=500, width=1000, title_text="Point comparison",autosize = False)
fig.update_traces(marker=dict(
        color='#007bbf', size=10, line_width=2))

fig.update_traces(line_color='black', line_width=2)

fig.show()

In [ ]:
#Doubt 2
'''
x, y = [], []
for x1, y1 in m1:
    x.append(x1)
    y.append(y1)
np.cov(x, y)
'''

In [ ]:
#Doubt 3
'''
x, y = [], []
for x1, y1 in m2:
    x.append(x1)
    y.append(y1)
np.cov(x, y)
'''

In [ ]:
m1, p = modify_const_Z(7,data[str(distances[15]*80)]['Train'],mean_source,cov_source,mean_target,cov_target,target = data[str(distances[22]*80)]['Train'], return_plots=False)
m2, p = modify_const_GD(7,data[str(distances[15]*80)]['Train'],M,b,target = data[str(distances[22]*80)]['Train'], return_plots=False)

In [ ]:
  fig = make_subplots(rows=1, cols=3, shared_xaxes = True, shared_yaxes = True
                      , subplot_titles=['Z-score', 'Gradient Descent', 'Target distribution'], horizontal_spacing= 0.02)

  
  fig.add_traces(
    [m1, m2, p], rows=[1,1,1], cols=[1,2,3]
  )

  fig.update_layout(height=420, width=1000, autosize = False)
  
  fig.update_xaxes(range=[-4.1, -1.9])
  fig.update_yaxes(range=[1.9, 4.1]) 
  fig.show()

In [ ]:
def L2dist(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

In [ ]:
method = "Z"

quadrants = [1, 7, 10, 15]
first = True

for i in tqdm(range(25)):
    print("i = ",i)
    for j in range(i+1,25):
        print(" j = ",j)
        source = data_new[str(distances[i]*80)]['Train']
        target = data_new[str(distances[j]*80)]['Train']
        source_test = data_new[str(distances[i]*80)]['Test']

        source_test2 = source_test.copy()

        for quadrant in quadrants:
            print("   quadrant = ", quadrant)
            if method == "GD":
              M, b = compute_parameters(quadrant, 'GD', source, target)
              new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
            elif method == "Z":
              mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target)
              new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)

            '''
            new_points2,indexes2 = modify_const_GD(quadrant, source, M, b)
            target3,indexes3 = modify_const_GD(quadrant, target, np.array([[1,0],[0,1]]), np.array([[0],[0]]))
            print("NP1")
            print(compute_mean_and_cov(new_points))
            print("NP2")
            print(compute_mean_and_cov(new_points2))
            print("T3")
            print(compute_mean_and_cov(target3))
            '''
            for k in range(len(indexes)):
                source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])
        
        F=[]
        for k in range(source_test2.shape[0]):
            data2=list(source_test2.iloc[k,:])
            data2=[[float(d.real), float(d.imag)] for d in data2]
            gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
            mus=gmm.means_
            sigmas=gmm.covariances_
            features=[distances[j]*80]
            for z in selCP_pos:
                mindist=None
                k_inc=None
                for w in range(16):
                    d=L2dist(mus[w],z)
                    if mindist is None or mindist>d:
                        mindist=d
                        k_inc=w
                #print(k_inc)
                #print(mus[k_inc])
                covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                #features = [*features, *mus[k_inc], *[covmat[0], covmat[3]]]
                features = [*features, *mus[k_inc], *covmat]
            F.append(features)

        header=['dist']
        
        for j in selCP:
            header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
            #header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ii_'+str(j)]]
        with open(path+'/modifiedData/'+method+'/alpha0.1.csv', 'a', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            # write the header
            if first:
              writer.writerow(header)
              first = False
            # write multiple rows
            writer.writerows(F)

In [ ]:
source = data[str(distances[22]*80)]['Train']
target = data[str(distances[24]*80)]['Train']
source_test = data[str(distances[22]*80)]['Test']
target_test = data[str(distances[24]*80)]['Test']

In [ ]:
m,b = compute_parameters(7, 'GD', source, target, log = True)

# Comparing methods

In [ ]:
l_gd = []
l_z = []
const = [1,7,10,15]

for i in const:
  p_gd,t = modify_const(i,"GD",source,target,plot = False)
  p_z,_ = modify_const(i,"Z",source,target,plot = False)
  aux_gd = [[x[0][0],x[1][0]] for x in p_gd]
  aux_z = [[x[0][0],x[1][0]] for x in p_z]
  m_gd,cov_gd = compute_mean_and_cov(aux_gd)
  m_z,cov_z = compute_mean_and_cov(aux_z)
  m_t,cov_t = compute_mean_and_cov(t)

  l_gd.append(total_loss(m_gd,m_t,cov_gd,cov_t))
  l_z.append(total_loss(m_z,m_t,cov_z,cov_t))

In [ ]:
loss_comparison = pd.DataFrame({"Const_P":[0],"Loss":[0],"Method":[""]})

for i in range(len(l_gd)):
  loss_comparison.loc[i] = [const[i],l_gd[i],"GD"]

for i in range(len(l_z)):
  loss_comparison.loc[i+4] = [const[i],l_z[i],"Z"]

In [ ]:
fig = px.bar(loss_comparison, x="Const_P", y="Loss", color='Method')
fig.show()